# Parte 3 - Reporting

- Empleando Python 3 generar un fichero Excel que contenga tres pestañas.
- Para este ejercicio las transformaciones queremos que se hagan en Pandas y no una
sentencia SQL
- Eplea el método/libreria de tu preferencia para generar el fichero Excel
- La información que queremos que se muestre en cada una de las pestañas es una
agrupación por mes y tipo de dia:
    - MES: Tendrá el formato año-mes, por ejemplo 2020-01
    -Tipo_dia: Tendrá el valor 1 si el viaje se realizó entre el lunes y el viernes
(ambos inclusive) y 2 si se hizo en fin de semana
    - Personas transportadas: El total de personas que han usado el servicio de Taxis
para ese mes y tipo_dia
    - Millas recorridas: El total de millas recorridas para ese mes y tipo_dia
    - Total servicios: El total de servicios que realizados para ese mes y tipo_dia

- Pestaña JFK: Tendrá la agrupación descrita para todos los viajes que tengan un 2 en la
variable RateCodeID
- Pestaña regular: Tendrá la agrupación descrita para todos los viajes que tengan un 1 en
la variable RateCodeID
- Pestaña Others: Tendrá la agrupación descrita para todos los viajes que no tengan ni un
1 ni un 2 en la variable RateCodeID

In [ ]:
###########################################################################################################

# Tools 

In [2]:
import pandas as pd
import openpyxl

# Data loading

In [ ]:
name = "../2020_yellow_jan_feb_mar.csv"
jan_feb_mar = pd.read_csv(name)

In [ ]:
jan_feb_mar

# Data manipulation

In [ ]:
month_col = 'month'
start_date_col = 'tpep_pickup_datetime'
final_date_col = 'tpep_dropoff_datetime'
passenger_col = 'passenger_count'
distance_col = 'trip_distance'
ratecode_col = 'RatecodeID'

df = jan_feb_mar.copy()

df = df[[month_col, start_date_col, final_date_col, passenger_col, distance_col, ratecode_col]]
df = df[(df[start_date_col] > '2020-01-01') & (df[start_date_col] <= '2020-03-31')]

type_object = df.select_dtypes(include='object').columns.to_list()
for elem in type_object:
        df[f"{elem}"] = pd.to_datetime(df[f"{elem}"], yearfirst=True, format="%Y/%m/%d %H:%M:%S")
        
df['day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek
df['tipo_dia'] = df['day_of_week'].apply(lambda x: 1 if x in range(0,5) else 2)
df['total_servicios'] = 1
df['mes'] = df['tpep_pickup_datetime'].dt.strftime('%Y-%m')

df.rename(columns={"trip_distance": "millas_recorridas",
                   "passenger_count":"personas_transportadas"}, 
          inplace=True)


In [ ]:
df

In [ ]:
table = df.copy()
table = table[['mes', 'tipo_dia', 'personas_transportadas', 'millas_recorridas', 'total_servicios', 'RatecodeID']]
table

- Pestaña JFK: Tendrá la agrupación descrita para todos los viajes que tengan un 2 en la
variable RateCodeID

In [ ]:
# Agrupamos por las variables indicadas
jfk = table.copy()
jfk = jfk.groupby(['mes', 'tipo_dia', 'RatecodeID']).sum().reset_index()
jfk

In [ ]:
# Filtramos RaatecodeID = 2
jfk = jfk[jfk['RatecodeID'] == 2]
jfk

In [ ]:
jfk.drop(columns=['RatecodeID'], inplace=True)
jfk

- Pestaña regular: Tendrá la agrupación descrita para todos los viajes que tengan un 1 en
la variable RateCodeID

In [ ]:
regular = table.copy()
regular = regular.groupby(['mes', 'tipo_dia', 'RatecodeID']).sum().reset_index()
regular

In [ ]:
regular = regular[regular['RatecodeID'] == 1]
regular

In [ ]:
regular.drop(columns=['RatecodeID'], inplace=True)
regular

- Pestaña Others: Tendrá la agrupación descrita para todos los viajes que no tengan ni un
1 ni un 2 en la variable RateCodeID

In [ ]:
others = table.copy()
others = others.groupby(['mes', 'tipo_dia', 'RatecodeID']).sum().reset_index()
others

In [ ]:
others = others[(others['RatecodeID']) != 1 & (others['RatecodeID'] != 2)]
others

In [ ]:
others.drop(columns=['RatecodeID'], inplace=True)
others

# Reporting

In [ ]:
with pd.ExcelWriter('report.xlsx') as writer:  
    jfk.to_excel(writer, sheet_name='JFK')
    regular.to_excel(writer, sheet_name='Regular')
    others.to_excel(writer, sheet_name='Others')